In [ ]:
# Imports
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)

from src import *
import scipy.sparse.linalg as spsl
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
# Hamiltonian Parameters
num_qubits = 15
J = 1
h = 1

# Algorithm Parameters
kd_ratio = 2.5
noise_threshold = 1e-2
epsilon = 1e-3
delta_t = 0.08
num_modmd_observables = 7
num_trials = 20

# Analysis Parameters
kstar_values = [500,1500,2500]
num_predicted_steps = 200

In [ ]:
# Generate Hamiltonian
sparse_hamiltonian = tfim_hamiltonian(num_qubits,J,h).to_matrix(sparse=True)

In [ ]:
# Construct reference state
indices = np.argsort(sparse_hamiltonian.diagonal())
reference_state = bitstring_superposition_state(num_qubits,[bin(indices[i])[2:] for i in range(6)])

# Get evolved reference states
max_K = int(np.round((kstar_values[-1]+num_predicted_steps+1)/(1+1/kd_ratio)))
max_d = int((max_K)/kd_ratio)
time_evolution_operator = -1j*sparse_hamiltonian*delta_t
evolved_reference_states = spsl.expm_multiply(time_evolution_operator,reference_state,start=0,stop=max_d+max_K+1,num = max_d+max_K+2)

In [ ]:
# Get MODMD observables
modmd_observables = [SparsePauliOp('I'*num_qubits).to_matrix(sparse=True)] + random_one_local_paulis(num_qubits,num_modmd_observables-1)

In [ ]:
# True/Measured signals
measured_signals = {idx:{kstar:[] for kstar in kstar_values} for idx in range(len(modmd_observables))}

for idx in range(len(modmd_observables)):

    odmd_observables = [modmd_observables[idx]]
    X_elements = generate_X_elements(odmd_observables,max_d,max_K,reference_state,evolved_reference_states)

    for kstar in kstar_values:
        for trial in range(num_trials):

            gaussian_noise = np.random.normal(0,epsilon,size=X_elements.shape) + 1j * np.random.normal(0,epsilon,size=X_elements.shape)
            noisy_X_elements = X_elements + gaussian_noise
            
            # Measure to some K*, extrapolate the rest
            K = int(np.round(kstar/(1+1/kd_ratio)))
            d = int(K/kd_ratio)

            k_range = range(K+d+1,K+d+1+num_predicted_steps)
            measured_signals[idx][kstar].append(noisy_X_elements[k_range])

In [ ]:
# MODMD Results
X_elements = generate_X_elements(modmd_observables,max_d,max_K,reference_state,evolved_reference_states)

predicted_signals_modmd = {kstar:[] for kstar in kstar_values}

for kstar in kstar_values:
    
    for trial in range(num_trials):

        gaussian_noise = np.random.normal(0,epsilon,size=X_elements.shape) + 1j * np.random.normal(0,epsilon,size=X_elements.shape)
        noisy_X_elements = X_elements + gaussian_noise

        # Measure to some K*, extrapolate the rest
        K = int(np.round(kstar/(1+1/kd_ratio)))
        d = int(K/kd_ratio)

        k_range = range(K+d+1,K+d+1+num_predicted_steps)

        X,Xp = X_matrices(len(modmd_observables),d,K,noisy_X_elements)
        A = A_matrix(noise_threshold,X,Xp,False)
        last_column = Xp[:,-1]

        pred = []
        for predicted_step in range(num_predicted_steps):
            last_column = A@last_column
            pred.append(last_column[-len(modmd_observables):])

        predicted_signals_modmd[kstar].append(pred)

In [ ]:
# Results processing
average_measured_signals = {}
for idx in range(len(modmd_observables)):
    for kstar in kstar_values:
        average_measured_signals[(idx,kstar)] = np.average(measured_signals[idx][kstar],0)

average_predicted_signals_modmd = {kstar:np.average(predicted_signals_modmd[kstar],0) for kstar in kstar_values}

In [ ]:
amplitude_type = 'Re'
idx = 5

colors = get_color_set('TFIM')
fig, ax = plt.subplots(len(kstar_values),1,figsize=(15,7),layout='constrained',sharey=True)
matplotlib.rcParams.update({'font.size': 14})

for i, kstar in enumerate(kstar_values):

    if amplitude_type == 'Re':
        ax[i].plot(range(1,num_predicted_steps+1),average_measured_signals[(idx,kstar)].real, 
                   label = 'True Signal' if i == 0 else None, color = colors[0])
        ax[i].plot(range(1,num_predicted_steps+1), average_predicted_signals_modmd[kstar][:,idx].real, '--', 
                   label = f'MODMD Predicted Signal' if i == 0 else None, color = colors[3])
    
    elif amplitude_type == 'Im':
        ax[i].plot(range(1,num_predicted_steps+1),average_measured_signals[(idx,kstar)].imag, 
                   label = 'True Signal' if i == 0 else None, color = colors[0])
        ax[i].plot(range(1,num_predicted_steps+1), average_predicted_signals_modmd[kstar][:,idx].imag, '--', 
                   label = f'MODMD Predicted Signal' if i == 0 else None, color = colors[3])

    ax[i].set_ylabel(f'k*={kstar}', rotation = -90, labelpad = 15)
    ax[i].yaxis.set_label_position("right")

fig.supxlabel('$k - k^*$')
fig.supylabel(rf'{amplitude_type}[$\langle\phi_0|O_i|\phi_0(k\Delta t)\rangle$]')
ax[0].legend( loc='upper right')